In [1]:
# 🔹 Mettre à jour git si nécessaire
!apt-get -qq install git

# 🔹 Cloner le dépôt GitHub
!git clone https://github.com/amitrj540/recommendation-engine.git

# 🔹 Afficher le contenu
!ls -R recommendation-engine


Cloning into 'recommendation-engine'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 318 (delta 1), reused 0 (delta 0), pack-reused 309 (from 1)
Receiving objects: 100% (318/318), 31.81 MiB | 22.60 MiB/s, done.
Resolving deltas: 100% (146/146), done.
recommendation-engine:
data		 LICENSE  models     recommendation_filters
data_processing  main.py  README.md  requirements.txt

recommendation-engine/data:
asin_title.json.gz  processed  raw

recommendation-engine/data/processed:
processed_data_here

recommendation-engine/data/raw:
All_Beauty_25.json.gz  meta_All_Beauty_25.json.gz  raw_data_here

recommendation-engine/data_processing:
data_cleaning.py  data_merge.py  feature_genration.py  text_processing.py

recommendation-engine/models:
collaborative_item_item.py    lin_svc.py  pickle_files
collaborative_model_based.py  nb.py	  sampler_sentiment_generator.py

recommendation-engine/models/pi

In [3]:
import pandas as pd

reviews_path = "/content/recommendation-engine/data/raw/All_Beauty_25.json.gz"
meta_path    = "/content/recommendation-engine/data/raw/meta_All_Beauty_25.json.gz"

reviews = pd.read_json(reviews_path)
meta    = pd.read_json(meta_path)

print("Reviews shape :", reviews.shape)
print("Meta shape    :", meta.shape)


Reviews shape : (92836, 12)
Meta shape    : (8223, 18)


In [4]:
reviews.head(3)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1,True,"04 11, 2017",A1FRUNW7IJMRCM,B019S2ED4O,Jennifer Risch,I used the whole product and seen absolutely n...,No results,1491868800,2,None,None
1,5,True,"09 18, 2015",A1D0AQT5685Q,B00ZASNPGI,terra jo,I love this pallet. I use it daily and will d...,I love this pallet,1442534400,None,None,None
2,5,True,"03 18, 2018",A5D9KWBEXXS37,B01DKQAXC0,Carlos M.,Wow!! I'm a cyclist and I swear a lot!!! You c...,Wow!! I'm a cyclist and I swear a ...,1521331200,None,{'Scent Name:': ' Original Essence'},None


In [5]:
reviews_clean = reviews[[
    "reviewerID",
    "asin",
    "overall",
    "unixReviewTime",
    "reviewText",
    "verified"
]].copy()


In [6]:
reviews_clean["reviewText"] = reviews_clean["reviewText"].fillna("")
reviews_clean = reviews_clean[reviews_clean["reviewText"].str.strip() != ""]


In [7]:
# rating valide
reviews_clean = reviews_clean[
    reviews_clean["overall"].between(1, 5)
]

# timestamps valides
reviews_clean = reviews_clean[
    reviews_clean["unixReviewTime"].notna()
]


In [8]:
reviews_clean = reviews_clean[reviews_clean["verified"] == True]


In [9]:
reviews_clean = reviews_clean.rename(columns={
    "reviewerID": "user_id",
    "asin": "item_id",
    "overall": "rating",
    "unixReviewTime": "ts",
    "reviewText": "review_text"
})


In [10]:
print("Reviews nettoyées :", reviews_clean.shape)


Reviews nettoyées : (80433, 6)


In [11]:
meta_clean = meta[[
    "asin",
    "title",
    "description",
    "feature",
    "brand",
    "category",
    "main_cat",
    "image",
    "price",
    "rank"
]].copy()


In [12]:
meta_clean["title"] = meta_clean["title"].fillna("")
meta_clean["description"] = meta_clean["description"].fillna("")

meta_clean["feature"] = meta_clean["feature"].apply(
    lambda x: " ".join(x) if isinstance(x, list) else ""
)

meta_clean["brand"] = meta_clean["brand"].fillna("unknown")


In [13]:
# garder UNE seule image
meta_clean["image"] = meta_clean["image"].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
)

# supprimer produits sans image
meta_clean = meta_clean[meta_clean["image"].notna()]


In [14]:
meta_clean["main_cat"] = meta_clean["main_cat"].fillna("Beauty")
meta_clean["category"] = meta_clean["category"].apply(
    lambda x: x if isinstance(x, list) else []
)


In [15]:
meta_clean = meta_clean.drop_duplicates("asin")


In [17]:
def to_text(x):
    if isinstance(x, list):
        return " ".join([str(i) for i in x])
    if isinstance(x, dict):
        return " ".join([f"{k} {v}" for k, v in x.items()])
    if pd.isna(x):
        return ""
    return str(x)


In [18]:
text_cols = ["title", "description", "feature", "brand"]

for col in text_cols:
    meta_clean[col] = meta_clean[col].apply(to_text)


In [19]:
meta_clean["item_text"] = (
    meta_clean["title"] + " " +
    meta_clean["description"] + " " +
    meta_clean["feature"] + " " +
    meta_clean["brand"]
)


In [20]:
meta_clean["item_text"].head(3)


,item_text
0,Devon Medical Carrying Case for Fingertip Puls...
1,Great Offer Set Kit of 3pcs High Quality Stret...
2,100pcs Silver Tone Hypo Allergenic Clutch Earr...


ÉTAPE 6 — SBERT : EMBEDDINGS DES PRODUITS (OBLIGATOIRE)

In [22]:
!pip install -q sentence-transformers


In [23]:
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
item_texts = meta_clean["item_text"].tolist()

item_embeddings = sbert_model.encode(
    item_texts,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True
)


Batches:   0%|          | 0/65 [00:00<?, ?it/s]

In [25]:
import numpy as np

np.save(
    "/content/recommendation-engine/data/processed/item_embeddings.npy",
    item_embeddings
)

meta_clean[["asin"]].to_csv(
    "/content/recommendation-engine/data/processed/item_ids.csv",
    index=False
)


In [26]:
item_embeddings.shape


(4159, 384)

ÉTAPE 7 — SBERT : PROFIL SÉMANTIQUE UTILISATEUR (TRÈS IMPORTANT)

In [27]:
user_text = (
    reviews_clean
    .groupby("user_id")["review_text"]
    .apply(lambda x: " ".join(x))
    .reset_index()
)


In [28]:
user_embeddings = sbert_model.encode(
    user_text["review_text"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True
)


Batches:   0%|          | 0/2421 [00:00<?, ?it/s]

In [29]:
np.save(
    "/content/recommendation-engine/data/processed/user_embeddings.npy",
    user_embeddings
)

user_text[["user_id"]].to_csv(
    "/content/recommendation-engine/data/processed/user_ids.csv",
    index=False
)


ÉTAPE 8 — SIMILARITÉ TEXTE (BASE DU COLD-START)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

# exemple : user 0
u_vec = user_embeddings[0].reshape(1, -1)

scores = cosine_similarity(u_vec, item_embeddings)[0]
top_items = scores.argsort()[-10:][::-1]


In [31]:
meta_clean.iloc[top_items][["asin", "title"]]


,asin,title
1796,B00N5K48R8,Cinthol Cool Cooling deo Talc 300 g
5852,B00J8MUS6W,Coolture Men's Cooling Vest
2661,B01FKJIGBU,"Onteck Laptop Sleeve,Waterproof Canvas Fabric ..."
6702,B00005GNB4,Cool Summer Collection
4623,B011J6LCCS,Ban Total Refresh Cooling Body Cloths 10 Count...
6917,B01D53MOS4,J Cat Soft Matte Lip Cooler 105 Peach Fruzzies
1787,B01G96YW16,"Desert Breeze Conditioner, Travel Size Hotel A..."
6659,B00S6SA7WY,LAND ROVER DISCOVERY2 / RANGE ROVER P38 OEM CO...
3831,B00115PM9C,Infrared Sauna Dome DS689 - Amazing Far Infrar...
4182,B00UKHHBFA,Cool Gear Go-Gear Silicone Squeezable Travel T...


ÉTAPE 9 — HYBRIDATION ALS + SBERT (CŒUR DU PROJET

In [33]:
def hybrid_score(als_score, sbert_score, alpha=0.7):
    return alpha * als_score + (1 - alpha) * sbert_score


evaluation

In [35]:
# On part des données nettoyées
# reviews_clean : user_id, item_id, rating, ts, review_text
# meta_clean    : asin, item_text, image, etc.

events = reviews_clean[["user_id", "item_id", "rating", "ts"]].copy()

# feedback implicite pour ALS
events["interaction"] = 1

print("Events shape:", events.shape)
events.head()


Events shape: (80433, 5)


,user_id,item_id,rating,ts,interaction
0,A1FRUNW7IJMRCM,B019S2ED4O,1,1491868800,1
1,A1D0AQT5685Q,B00ZASNPGI,5,1442534400,1
2,A5D9KWBEXXS37,B01DKQAXC0,5,1521331200,1
3,A2SZ3QD6YRPZSK,B00NT0AR7E,5,1449100800,1
4,A1MUHED57OHY3M,B01G53GFKO,4,1483401600,1


In [36]:
assert "ts" in events.columns
assert "user_id" in events.columns
assert "item_id" in events.columns


In [38]:
events_sorted = events.sort_values("ts")

train = []
test = []

for u, df_u in events_sorted.groupby("user_id"):
    if len(df_u) < 2:   # ⬅️ CHANGEMENT ICI
        continue
    split = max(1, int(len(df_u) * 0.8))
    train.append(df_u.iloc[:split])
    test.append(df_u.iloc[split:])

train = pd.concat(train)
test  = pd.concat(test)

print("Train:", train.shape)
print("Test :", test.shape)
print("Users train:", train.user_id.nunique())
print("Users test :", test.user_id.nunique())


Train: (2981, 5)
Test : (2701, 5)
Users train: 2699
Users test : 2699


In [39]:
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
item_enc = LabelEncoder()

train["user_idx"] = user_enc.fit_transform(train["user_id"])
train["item_idx"] = item_enc.fit_transform(train["item_id"])

test = test[
    test["user_id"].isin(user_enc.classes_) &
    test["item_id"].isin(item_enc.classes_)
].copy()

test["user_idx"] = user_enc.transform(test["user_id"])
test["item_idx"] = item_enc.transform(test["item_id"])


In [40]:
from scipy.sparse import csr_matrix

n_users = train.user_idx.nunique()
n_items = train.item_idx.nunique()

train_matrix = csr_matrix(
    (train["interaction"], (train["user_idx"], train["item_idx"])),
    shape=(n_users, n_items)
)


In [44]:
!pip install -q implicit

import implicit
from implicit.als import AlternatingLeastSquares

als = AlternatingLeastSquares(
    factors=64,
    regularization=0.01,
    iterations=20,
    random_state=42
)

als.fit(train_matrix)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


  0%|          | 0/20 [00:00<?, ?it/s]

In [45]:
import numpy as np
import math

def ndcg_at_k(ranked_items, ground_truth, k):
    score = 0.0
    for i, item in enumerate(ranked_items[:k]):
        if item in ground_truth:
            score += 1 / math.log2(i + 2)
    ideal = sum(1 / math.log2(i + 2) for i in range(min(len(ground_truth), k)))
    return score / ideal if ideal > 0 else 0.0


def evaluate_model(recommend_fn, K=10):
    P, R, N = [], [], []

    for u in test.user_idx.unique():
        truth = set(test[test.user_idx == u].item_idx)
        if not truth:
            continue

        recs = recommend_fn(u, K)
        hit = len(set(recs) & truth)

        P.append(hit / K)
        R.append(hit / len(truth))
        N.append(ndcg_at_k(recs, truth, K))

    return np.mean(P), np.mean(R), np.mean(N)


In [50]:
def recommend_als(user_idx, K):
    item_ids, scores = als.recommend(
        user_idx,
        train_matrix[user_idx],
        N=K,
        filter_already_liked_items=True
    )
    return list(item_ids)


In [51]:
print(recommend_als(0, 10))


[np.int32(701), np.int32(406), np.int32(557), np.int32(904), np.int32(9), np.int32(253), np.int32(126), np.int32(98), np.int32(217), np.int32(33)]


In [52]:
p10, r10, n10 = evaluate_model(recommend_als, K=10)
print(f"ALS @10 → Precision={p10:.4f}, Recall={r10:.4f}, NDCG={n10:.4f}")


ALS @10 → Precision=0.0078, Recall=0.0779, NDCG=0.0502


In [53]:
def recommend_hybrid(user_idx, K, alpha=0.7):
    item_ids, als_scores = als.recommend(
        user_idx,
        train_matrix[user_idx],
        N=200,
        filter_already_liked_items=True
    )

    scores = {}
    u_vec = user_embeddings[user_idx]

    for item_idx, als_score in zip(item_ids, als_scores):
        asin = item_enc.inverse_transform([item_idx])[0]
        if asin not in item_id_to_idx:
            continue

        sbert_score = cosine_similarity(
            u_vec.reshape(1, -1),
            item_embeddings[item_id_to_idx[asin]].reshape(1, -1)
        )[0][0]

        scores[item_idx] = alpha * als_score + (1 - alpha) * sbert_score

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [i for i, _ in ranked[:K]]


In [54]:
p10, r10, n10 = evaluate_model(recommend_hybrid, K=10)
print(f"HYBRID @10 → Precision={p10:.4f}, Recall={r10:.4f}, NDCG={n10:.4f}")


HYBRID @10 → Precision=0.0012, Recall=0.0124, NDCG=0.0057


In [55]:
train_items = train.groupby("user_idx")["item_idx"].apply(set).to_dict()


In [56]:
def adaptive_alpha(user_idx):
    n = len(train_items.get(user_idx, []))
    if n < 2:
        return 0.3   # cold user → contenu
    elif n < 5:
        return 0.5
    else:
        return 0.8   # user actif → ALS


In [57]:
def get_als_candidates(user_idx, N=300):
    item_ids, scores = als.recommend(
        user_idx,
        train_matrix[user_idx],
        N=N,
        filter_already_liked_items=True
    )
    return item_ids, scores


In [58]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def content_score(user_idx):
    seen = train_items.get(user_idx, set())
    if len(seen) == 0:
        return None

    user_vec = item_embeddings[list(seen)].mean(axis=0)
    scores = cosine_similarity(
        user_vec.reshape(1, -1),
        item_embeddings
    )[0]
    return scores


In [59]:
def recommend_hybrid_adaptive(user_idx, K=10):
    seen = train_items.get(user_idx, set())
    alpha = adaptive_alpha(user_idx)

    als_items, als_scores = get_als_candidates(user_idx, N=300)
    als_map = dict(zip(als_items, als_scores))

    content_scores = content_score(user_idx)

    final_scores = {}

    for i in als_items:
        if i in seen:
            continue

        s_als = als_map.get(i, 0.0)
        s_cont = content_scores[i] if content_scores is not None else 0.0

        final_scores[i] = alpha * s_als + (1 - alpha) * s_cont

    ranked = sorted(final_scores, key=final_scores.get, reverse=True)
    return ranked[:K]


In [60]:
item_popularity = train["item_idx"].value_counts().to_dict()

def personalized_rerank(user_idx, K=10):
    seen = train_items.get(user_idx, set())
    n = len(seen)

    if n < 2:
        alpha_pop = 1.5
    elif n < 5:
        alpha_pop = 1.0
    else:
        alpha_pop = 0.5

    recs = recommend_hybrid_adaptive(user_idx, K=300)
    scores = {}

    for i in recs:
        pop = np.log1p(item_popularity.get(i, 1)) ** alpha_pop
        scores[i] = 1 / pop

    return sorted(scores, key=scores.get, reverse=True)[:K]


In [61]:
p10, r10, n10 = evaluate_model(personalized_rerank, K=10)
print(f"Adaptive Hybrid @10 → Precision={p10:.4f}, Recall={r10:.4f}, NDCG={n10:.4f}")


Adaptive Hybrid @10 → Precision=0.0001, Recall=0.0005, NDCG=0.0002


ALS + Time-aware + Interaction Weighting

In [64]:
interaction = rating × time_decay × verified × helpful_votes


SyntaxError: invalid character '×' (U+00D7) (ipython-input-219761396.py, line 1)

In [65]:
import numpy as np
import pandas as pd

df = reviews_clean.copy()

# sécurité
if "helpful_vote" not in df.columns:
    df["helpful_vote"] = 0

df["helpful_vote"] = df["helpful_vote"].fillna(0).astype(int)
df["verified"] = df["verified"].astype(int)

# --- Time decay ---
t_max = df["ts"].max()
LAMBDA = 1e-7  # valeur standard

df["time_weight"] = np.exp(-LAMBDA * (t_max - df["ts"]))

# --- Interaction weight FINAL ---
df["interaction_weight"] = (
    df["rating"]
    * (1 + np.log1p(df["helpful_vote"]))
    * df["verified"].clip(lower=1)
    * df["time_weight"]
)

df[["rating", "helpful_vote", "verified", "time_weight", "interaction_weight"]].head()


,rating,helpful_vote,verified,time_weight,interaction_weight
0,1,0,1,0.009661,0.009661
1,5,0,1,0.000070,0.000348
2,5,0,1,0.183886,0.919429
3,5,0,1,0.000134,0.000671
4,4,0,1,0.004143,0.016571


In [66]:
df = df.sort_values("ts")

train, test = [], []

for u, g in df.groupby("user_id"):
    if len(g) < 2:
        continue
    split = max(1, int(len(g) * 0.8))
    train.append(g.iloc[:split])
    test.append(g.iloc[split:])

train = pd.concat(train)
test  = pd.concat(test)

print("Train:", train.shape)
print("Test :", test.shape)


Train: (2981, 9)
Test : (2701, 9)


In [67]:
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
item_enc = LabelEncoder()

train["user_idx"] = user_enc.fit_transform(train["user_id"])
train["item_idx"] = item_enc.fit_transform(train["item_id"])

test = test[
    test["user_id"].isin(user_enc.classes_) &
    test["item_id"].isin(item_enc.classes_)
].copy()

test["user_idx"] = user_enc.transform(test["user_id"])
test["item_idx"] = item_enc.transform(test["item_id"])


In [68]:
from scipy.sparse import csr_matrix

n_users = train.user_idx.nunique()
n_items = train.item_idx.nunique()

train_matrix = csr_matrix(
    (
        train["interaction_weight"],
        (train["user_idx"], train["item_idx"])
    ),
    shape=(n_users, n_items)
)

train_matrix.shape


(2699, 1061)

In [69]:
from implicit.als import AlternatingLeastSquares

als_weighted = AlternatingLeastSquares(
    factors=64,
    regularization=0.05,
    iterations=30,
    random_state=42
)

als_weighted.fit(train_matrix)


  0%|          | 0/30 [00:00<?, ?it/s]

In [70]:
import math

def ndcg_at_k(ranked, truth, k):
    dcg = sum(
        1 / math.log2(i + 2)
        for i, item in enumerate(ranked[:k])
        if item in truth
    )
    idcg = sum(
        1 / math.log2(i + 2)
        for i in range(min(len(truth), k))
    )
    return dcg / idcg if idcg > 0 else 0.0


def evaluate_model(recommend_fn, K=10):
    P, R, N = [], [], []

    for u in test.user_idx.unique():
        truth = set(test[test.user_idx == u].item_idx)
        if not truth:
            continue

        recs = recommend_fn(u, K)
        hit = len(set(recs) & truth)

        P.append(hit / K)
        R.append(hit / len(truth))
        N.append(ndcg_at_k(recs, truth, K))

    return np.mean(P), np.mean(R), np.mean(N)


In [71]:
def recommend_als_weighted(user_idx, K=10):
    item_ids, scores = als_weighted.recommend(
        user_idx,
        train_matrix[user_idx],
        N=K,
        filter_already_liked_items=True
    )
    return list(item_ids)


In [72]:
p10, r10, n10 = evaluate_model(recommend_als_weighted, K=10)

print(f"ALS Weighted @10 → Precision={p10:.4f}, Recall={r10:.4f}, NDCG={n10:.4f}")


ALS Weighted @10 → Precision=0.0171, Recall=0.1713, NDCG=0.0983


🚀 APPROCHE 2 — ALS + Popularity-aware Re-ranking (SANS contenu)

In [73]:
import numpy as np

# Popularité des items
item_pop = train["item_idx"].value_counts().to_dict()

def recommend_als_popularity(user_idx, K=10):
    item_ids, scores = als_weighted.recommend(
        user_idx,
        train_matrix[user_idx],
        N=200,
        filter_already_liked_items=True
    )

    final_scores = {}
    for i, s in zip(item_ids, scores):
        pop_penalty = np.log1p(item_pop.get(i, 1))
        final_scores[i] = s / pop_penalty

    ranked = sorted(final_scores, key=final_scores.get, reverse=True)
    return ranked[:K]


p, r, n = evaluate_model(recommend_als_popularity, K=10)
print(f"ALS + Pop Re-rank @10 → P={p:.4f}, R={r:.4f}, NDCG={n:.4f}")


ALS + Pop Re-rank @10 → P=0.0138, R=0.1383, NDCG=0.0763


🚀 APPROCHE 3 — Two-Stage Recommender (Industry-grade)
💡 Idée Amazon / Netflix

Stage 1 : ALS → candidats

Stage 2 : modèle ML (LightGBM)

In [74]:
!pip install -q lightgbm


In [79]:
import lightgbm as lgb

# ---------- Feature engineering ----------
item_stats = train.groupby("item_idx").agg(
    mean_rating=("rating", "mean"),
    verified_ratio=("verified", "mean"),
    popularity=("user_idx", "count")
).fillna(0)

def build_features(user_idx):
    item_ids, als_scores = als_weighted.recommend(
        user_idx,
        train_matrix[user_idx],
        N=300,
        filter_already_liked_items=True
    )

    rows = []
    for i, s in zip(item_ids, als_scores):
        # Ensure item_stats has the item_idx, or provide default values
        if i not in item_stats.index:
            # Handle items not present in item_stats by assigning default values
            # This case implies an item recommended by ALS was not in the training set
            # used for feature aggregation (which can happen, but should be rare if item_stats uses all train data).
            # For robustness, we assign common defaults or the overall mean/ratio.
            rows.append([
                s, # ALS score
                0.0, # default popularity log1p
                0.0, # default mean_rating
                0.0  # default verified_ratio
            ])
            continue

        stats = item_stats.loc[i]
        rows.append([
            s,
            np.log1p(stats["popularity"]),
            stats["mean_rating"],
            stats["verified_ratio"]
        ])
    return np.array(rows), item_ids


# ---------- Train a simple ranker ----------
X, y = [], []

for u in train.user_idx.unique()[:500]:  # subset suffisant
    feats, items = build_features(u)
    truth = set(test[test.user_idx == u].item_idx)
    labels = [1 if i in truth else 0 for i in items]

    # Only add if feats is not empty (i.e., some items were recommended and had features)
    if len(feats) > 0:
        X.extend(feats)
        y.extend(labels)

X = np.array(X)
y = np.array(y)

# Calculate scale_pos_weight for imbalanced data
pos_count = np.sum(y == 1)
neg_count = np.sum(y == 0)
scale_pos_weight_val = neg_count / pos_count if pos_count > 0 else 1 # Avoid division by zero

ranker = lgb.LGBMClassifier(n_estimators=100, max_depth=5, scale_pos_weight=scale_pos_weight_val)
ranker.fit(X, y)


# ---------- Recommendation ----------
def recommend_two_stage(user_idx, K=10):
    feats, items = build_features(user_idx)

    # If no features could be built (e.g., no ALS candidates found with features), return empty list
    if len(feats) == 0:
        return []

    scores = ranker.predict_proba(feats)[:, 1]
    ranked = [i for i, _ in sorted(zip(items, scores), key=lambda x: x[1], reverse=True)]
    return ranked[:K]


p, r, n = evaluate_model(recommend_two_stage, K=10)
print(f"Two-Stage @10 → P={p:.4f}, R={r:.4f}, NDCG={n:.4f}")

[LightGBM] [Info] Number of positive: 150, number of negative: 149850
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 346
[LightGBM] [Info] Number of data points in the train set: 150000, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001000 -> initscore=-6.906755
[LightGBM] [Info] Start training from score -6.906755
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

Two-Stage @10 → P=0.0221, R=0.2214, NDCG=0.1237


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

🚀 APPROCHE 4 — Cold-start : règles intelligentes
💡 Idée

Pas de ML → règles business intelligente

In [76]:
# Popularité récente
recent_items = (
    train.sort_values("ts", ascending=False)
    .groupby("item_idx")
    .size()
    .sort_values(ascending=False)
)

def recommend_cold_user(K=10):
    return list(recent_items.index[:K])


def recommend_mixed(user_idx, K=10):
    seen = train[train.user_idx == user_idx]
    if len(seen) < 2:
        return recommend_cold_user(K)
    return recommend_als_weighted(user_idx, K)


p, r, n = evaluate_model(recommend_mixed, K=10)
print(f"Cold-aware @10 → P={p:.4f}, R={r:.4f}, NDCG={n:.4f}")


Cold-aware @10 → P=0.0496, R=0.4964, NDCG=0.3072


🚀 APPROCHE 5 — Graph-based CF (Item–Item KNN)
💡 Idée

Similitude de co-achats → robuste au sparsity.

In [77]:
from implicit.nearest_neighbours import ItemItemRecommender

item_item = ItemItemRecommender(K=50)
item_item.fit(train_matrix.T)

def recommend_item_item(user_idx, K=10):
    item_ids, scores = item_item.recommend(
        user_idx,
        train_matrix[user_idx],
        N=K,
        filter_already_liked_items=True
    )
    return list(item_ids)


p, r, n = evaluate_model(recommend_item_item, K=10)
print(f"Item-Item KNN @10 → P={p:.4f}, R={r:.4f}, NDCG={n:.4f}")


/usr/local/lib/python3.12/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.00019311904907226562 seconds
  warnings.warn(


  0%|          | 0/2699 [00:00<?, ?it/s]

Item-Item KNN @10 → P=0.0128, R=0.1280, NDCG=0.0834


modele final apres ces approche if cold_user:
    use Cold-aware rules
else:
    use ALS Weighted


if cold_user:
    Cold-aware rules
else:
    ALS Weighted (time-aware)


In [81]:
from IPython.display import Image, display

# prendre un produit au hasard
row = meta_clean.sample(1).iloc[0]

print("ASIN :", row["asin"])
print("Titre :", row["title"])
print("Image URL :", row["image"])

display(Image(url=row["image"], width=250))


ASIN : B00IR50L9A
Titre : TINY 2MM PRESSCZ GEM STEEL LIP RING LABRET MONROE 16g INTERNAL THREA
Image URL : https://images-na.ssl-images-amazon.com/images/I/41eEwBwarXL._SS40_.jpg


In [82]:
from IPython.display import display, HTML

html = "<div style='display:flex; gap:20px;'>"

for _, row in meta_clean.sample(5).iterrows():
    html += f"""
    <div style="width:180px; text-align:center">
        <img src="{row['image']}" width="160"><br>
        <b>{row['title'][:60]}</b><br>
        <small>{row['brand']}</small>
    </div>
    """

html += "</div>"

display(HTML(html))


In [83]:
def show_recommendations(user_idx, K=5):
    rec_items = recommend_als_weighted(user_idx, K)
    items = meta_clean.iloc[rec_items]

    html = "<div style='display:flex; gap:20px;'>"
    for _, row in items.iterrows():
        html += f"""
        <div style="width:180px; text-align:center">
            <img src="{row['image']}" width="160"><br>
            <b>{row['title'][:60]}</b>
        </div>
        """
    html += "</div>"
    display(HTML(html))


# exemple
show_recommendations(user_idx=0, K=5)


ÉTAPE 1 — Définir les cold users (clair et juste)

In [86]:
COLD_THRESHOLD = 2  # standard et justifié

user_interactions = train.groupby("user_idx").size().to_dict()

def is_cold_user(user_idx):
    return user_interactions.get(user_idx, 0) < COLD_THRESHOLD


ÉTAPE 2 — Définir les deux stratégies

In [87]:
# Popularité récente globale
recent_items = (
    train.sort_values("ts", ascending=False)
    .groupby("item_idx")
    .size()
    .sort_values(ascending=False)
)

def recommend_cold(K=10):
    return list(recent_items.index[:K])


In [88]:
def recommend_warm(user_idx, K=10):
    item_ids, scores = als_weighted.recommend(
        user_idx,
        train_matrix[user_idx],
        N=K,
        filter_already_liked_items=True
    )
    return list(item_ids)


ÉTAPE 3 — MODÈLE FINAL COMBINÉ (CŒUR)

In [89]:
def recommend_final(user_idx, K=10):
    if is_cold_user(user_idx):
        return recommend_cold(K)
    else:
        return recommend_warm(user_idx, K)


🟢 ÉTAPE 4 — ÉVALUATION DU MODÈLE FINAL

In [90]:
p, r, n = evaluate_model(recommend_final, K=10)

print(f"FINAL MODEL @10 → Precision={p:.4f}, Recall={r:.4f}, NDCG={n:.4f}")


FINAL MODEL @10 → Precision=0.0496, Recall=0.4964, NDCG=0.3072


# decision final

onc il faut sauvegarder :

✅ ALS Weighted model

✅ Matrice sparse utilisateur–item

✅ Encoders (user_id ↔ user_idx, item_id ↔ item_idx)

✅ Statistiques cold-start

✅ Métadonnées produits (images, titres, etc.)

✅ Paramètres (thresholds, etc.)

In [91]:
import os

SAVE_DIR = "/content/recommender_model"
os.makedirs(SAVE_DIR, exist_ok=True)


ÉTAPE 2 — Sauvegarder le modèle ALS Weighted

In [92]:
import pickle

with open(f"{SAVE_DIR}/als_weighted.pkl", "wb") as f:
    pickle.dump(als_weighted, f)


🟢 ÉTAPE 3 — Sauvegarder la matrice sparse

In [93]:
from scipy.sparse import save_npz

save_npz(f"{SAVE_DIR}/train_matrix.npz", train_matrix)


🟢 ÉTAPE 4 — Sauvegarder les encoders

In [94]:
with open(f"{SAVE_DIR}/user_encoder.pkl", "wb") as f:
    pickle.dump(user_enc, f)

with open(f"{SAVE_DIR}/item_encoder.pkl", "wb") as f:
    pickle.dump(item_enc, f)


🟢 ÉTAPE 5 — Sauvegarder les infos cold-start

In [95]:
# nombre d'interactions par utilisateur
user_interactions = train.groupby("user_idx").size().to_dict()

with open(f"{SAVE_DIR}/user_interactions.pkl", "wb") as f:
    pickle.dump(user_interactions, f)

# popularité récente globale (pour cold users)
recent_items = (
    train.sort_values("ts", ascending=False)
    .groupby("item_idx")
    .size()
    .sort_values(ascending=False)
)

with open(f"{SAVE_DIR}/recent_items.pkl", "wb") as f:
    pickle.dump(list(recent_items.index), f)


ETAPE 6 — Sauvegarder les métadonnées produits (pour l’interface)

In [96]:
meta_for_app = meta_clean[[
    "asin",
    "title",
    "brand",
    "image",
    "price",
    "category",
    "main_cat"
]].copy()

meta_for_app.to_parquet(f"{SAVE_DIR}/items_metadata.parquet", index=False)


🟢 ÉTAPE 7 — Sauvegarder la config du modèle

In [99]:
config = {
    "cold_threshold": 2,
    "model_type": "ALS Weighted + Cold-aware",
    "topK": 10
}

with open(f"{SAVE_DIR}/config.pkl", "wb") as f:
    pickle.dump(config, f)


sauvegarder sur drive

In [100]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [101]:
import os

SAVE_DIR = "/content/drive/MyDrive/systeme_de_recommandation_final_asmae"
os.makedirs(SAVE_DIR, exist_ok=True)

print("Dossier de sauvegarde :", SAVE_DIR)


Dossier de sauvegarde : /content/drive/MyDrive/systeme_de_recommandation_final_asmae


In [102]:
import pickle

with open(os.path.join(SAVE_DIR, "als_weighted.pkl"), "wb") as f:
    pickle.dump(als_weighted, f)


In [103]:
from scipy.sparse import save_npz

save_npz(
    os.path.join(SAVE_DIR, "train_matrix.npz"),
    train_matrix
)


In [104]:
with open(os.path.join(SAVE_DIR, "user_encoder.pkl"), "wb") as f:
    pickle.dump(user_enc, f)

with open(os.path.join(SAVE_DIR, "item_encoder.pkl"), "wb") as f:
    pickle.dump(item_enc, f)


In [105]:
user_interactions = train.groupby("user_idx").size().to_dict()

with open(os.path.join(SAVE_DIR, "user_interactions.pkl"), "wb") as f:
    pickle.dump(user_interactions, f)


In [106]:
recent_items = (
    train.sort_values("ts", ascending=False)
    .groupby("item_idx")
    .size()
    .sort_values(ascending=False)
)

with open(os.path.join(SAVE_DIR, "recent_items.pkl"), "wb") as f:
    pickle.dump(list(recent_items.index), f)


In [107]:
meta_for_app = meta_clean[[
    "asin",
    "title",
    "brand",
    "image",
    "price",
    "category",
    "main_cat"
]].copy()

meta_for_app.to_parquet(
    os.path.join(SAVE_DIR, "items_metadata.parquet"),
    index=False
)


In [108]:
config = {
    "model_name": "ALS Weighted + Cold-aware",
    "cold_threshold": 2,
    "topK": 10,
    "dataset": "Amazon All_Beauty_25"
}

with open(os.path.join(SAVE_DIR, "config.pkl"), "wb") as f:
    pickle.dump(config, f)


In [109]:
os.listdir(SAVE_DIR)


['als_weighted.pkl',
 'train_matrix.npz',
 'user_encoder.pkl',
 'item_encoder.pkl',
 'user_interactions.pkl',
 'recent_items.pkl',
 'items_metadata.parquet',
 'config.pkl']

# Amelioration grok

🟢 AMÉLIORATION 1 — Grid Search ALS (Hyperparamètres)

📌 Gain attendu : +10–15 % NDCG
📌 Coût : faible (CPU only)

In [110]:
from implicit.als import AlternatingLeastSquares
import numpy as np

factors_list = [50, 100, 150]
regularization_list = [0.01, 0.05]
iterations_list = [15, 25]

best_score = 0
best_model = None
best_params = None

for f in factors_list:
    for reg in regularization_list:
        for it in iterations_list:
            model = AlternatingLeastSquares(
                factors=f,
                regularization=reg,
                iterations=it,
                random_state=42
            )
            model.fit(train_matrix)

            p, r, n = evaluate_model(
                lambda u, K=10: model.recommend(
                    u, train_matrix[u], N=K, filter_already_liked_items=True
                )[0],
                K=10
            )

            print(f"f={f}, reg={reg}, it={it} → NDCG={n:.4f}")

            if n > best_score:
                best_score = n
                best_model = model
                best_params = (f, reg, it)

print("\nBEST ALS:", best_params, "NDCG:", best_score)


  0%|          | 0/15 [00:00<?, ?it/s]

f=50, reg=0.01, it=15 → NDCG=0.0252


  0%|          | 0/25 [00:00<?, ?it/s]

f=50, reg=0.01, it=25 → NDCG=0.0274


  0%|          | 0/15 [00:00<?, ?it/s]

...

f=150, reg=0.01, it=25 → NDCG=0.0137


  0%|          | 0/15 [00:00<?, ?it/s]

f=150, reg=0.05, it=15 → NDCG=0.0694


  0%|          | 0/25 [00:00<?, ?it/s]

f=150, reg=0.05, it=25 → NDCG=0.0780

BEST ALS: (100, 0.05, 25) NDCG: 0.09387971530843711


AMÉLIORATION 2 — Cold-start par catégorie (TRÈS FORT)

📌 Gain attendu : +20 % recall cold users
📌 Inspiré Amazon Personalize

In [116]:
# 1️⃣ Récupérer les ASIN utilisés par le modèle
known_asins = set(item_enc.classes_)

# 2️⃣ Filtrer meta_clean sur les items connus
meta_known = meta_clean[meta_clean["asin"].isin(known_asins)].copy()

# 3️⃣ Ajouter item_idx uniquement pour ces items
meta_known["item_idx"] = item_enc.transform(meta_known["asin"])

# 4️⃣ Mapping item → catégorie
item_cat = meta_known.set_index("item_idx")["main_cat"].to_dict()

# 5️⃣ Popularité par catégorie (basée sur train)
cat_popularity = (
    train.assign(cat=train.item_idx.map(item_cat))
    .dropna(subset=["cat"])
    .groupby(["cat", "item_idx"])
    .size()
    .reset_index(name="cnt")
)

# 6️⃣ Recommandation cold-start par catégorie
def recommend_cold_by_category(last_item_idx=None, K=10):
    if last_item_idx is None:
        return recent_items[:K]

    cat = item_cat.get(last_item_idx)
    if cat is None:
        return recent_items[:K]

    items = (
        cat_popularity[cat_popularity.cat == cat]
        .sort_values("cnt", ascending=False)
        .item_idx.tolist()
    )
    return items[:K]


🟢 AMÉLIORATION 3 — Re-ranking avec métadonnées (SANS DL)

📌 Gain attendu : +5–10 % Precision
📌 ZÉRO GPU

In [118]:
# Features simples pour reranking
# Statistiques item pour reranking metadata
item_stats = (
    train.groupby("item_idx")
    .agg(
        popularity=("item_idx", "count"),   # nombre d'interactions
        avg_rating=("rating", "mean")       # moyenne des notes
    )
)


def rerank_metadata(items, K=10):
    scored = []
    for i in items:
        pop = item_stats.loc[i].popularity if i in item_stats.index else 0
        rating = item_stats.loc[i].avg_rating if i in item_stats.index else 3
        score = 0.7 * pop + 0.3 * rating
        scored.append((i, score))

    return [i for i, _ in sorted(scored, key=lambda x: x[1], reverse=True)[:K]]


🟢 AMÉLIORATION 4 — Content-based léger (TF-IDF)

📌 Gain attendu : +15 % NDCG sur sparse users
📌 CPU only

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

texts = meta_clean["title"].fillna("") + " " + meta_clean["brand"].fillna("")
tfidf = TfidfVectorizer(max_features=5000, stop_words="english")
X = tfidf.fit_transform(texts)

def content_similar_items(item_idx, K=10):
    idx = meta_clean.index.get_loc(item_idx)
    sims = cosine_similarity(X[idx], X)[0]
    top = sims.argsort()[::-1][1:K+1]
    return meta_clean.iloc[top].item_idx.tolist()


AMÉLIORATION 5 — Sentiment-weighted interactions (CPU)

📌 Inspiré des papiers Amazon Reviews
📌 ZÉRO GPU

In [121]:
# 1️⃣ Users & items connus du modèle
known_users = set(user_enc.classes_)
known_items = set(item_enc.classes_)

# 2️⃣ Filtrer les reviews compatibles avec le modèle
reviews_known = reviews[
    reviews["reviewerID"].isin(known_users) &
    reviews["asin"].isin(known_items)
].copy()

# 3️⃣ Encoder uniquement les labels connus
reviews_known["user_idx"] = user_enc.transform(reviews_known["reviewerID"])
reviews_known["item_idx"] = item_enc.transform(reviews_known["asin"])

# 4️⃣ Joindre le texte au train
train_with_text = train.merge(
    reviews_known[["user_idx", "item_idx", "reviewText"]],
    on=["user_idx", "item_idx"],
    how="left"
)

# 5️⃣ Sentiment analysis (VADER)
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download("vader_lexicon")

sia = SentimentIntensityAnalyzer()

train_with_text["sentiment"] = (
    train_with_text["reviewText"]
    .fillna("")
    .apply(lambda x: sia.polarity_scores(x)["compound"])
)

train_with_text.head()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,user_id,item_id,rating,ts,review_text,verified,helpful_vote,time_weight,interaction_weight,user_idx,item_idx,reviewText,sentiment
0,A030678248T9JPFWRHP5,B00H1M8R3C,5,1440806400,This is the best deal on nose rings I've found...,1,0,5.853299e-05,2.926650e-04,0,497,This is the best deal on nose rings I've found...,0.8849
1,A103ZEEXHH2KIA,B0017I8Z6K,5,1463356800,I ordered this because my daughter swims and p...,1,0,5.581516e-04,2.790758e-03,1,172,I ordered this because my daughter swims and p...,0.6249
2,A103ZEEXHH2KIA,B0017I8Z6K,5,1463356800,I ordered this because my daughter swims and p...,1,0,5.581516e-04,2.790758e-03,1,172,I ordered this because my daughter swims and p...,0.6249
3,A105LX18R3QR6S,B0014SQQ3M,1,1366761600,I used this product before and purchased from ...,1,0,3.561852e-08,3.561852e-08,2,156,I used this product before and purchased from ...,0.5574
4,A105LX18R3QR6S,B0014SQQ3M,1,1366761600,I used this product before and purchased from ...,1,0,3.561852e-08,3.561852e-08,2,156,I used this product before and purchased from ...,0.5574


evaluation

BASELINE — Modèle Final (RÉFÉRENCE)

In [122]:
def recommend_baseline(user_idx, K=10):
    if user_interactions.get(user_idx, 0) < 2:
        return recent_items[:K]
    items, _ = als_weighted.recommend(
        user_idx, train_matrix[user_idx],
        N=K, filter_already_liked_items=True
    )
    return list(items)

baseline_scores = evaluate_model(recommend_baseline, K=10)
baseline_scores


(np.float64(0.0048503611971104236),
 np.float64(0.048503611971104234),
 np.float64(0.0230332869759662))

🟢 AMÉLIORATION 1 — ALS hyperparamètres optimisés

In [123]:
def recommend_als_tuned(user_idx, K=10):
    if user_interactions.get(user_idx, 0) < 2:
        return recent_items[:K]
    items, _ = best_model.recommend(
        user_idx, train_matrix[user_idx],
        N=K, filter_already_liked_items=True
    )
    return list(items)

als_tuned_scores = evaluate_model(recommend_als_tuned, K=10)
als_tuned_scores


(np.float64(0.004747162022703819),
 np.float64(0.047471620227038186),
 np.float64(0.022754861363946284))

🟢 AMÉLIORATION 2 — Cold-start par catégorie

In [124]:
def recommend_cold_cat(user_idx, K=10):
    if user_interactions.get(user_idx, 0) < 2:
        user_items = train[train.user_idx == user_idx]
        last_item = user_items.sort_values("ts").item_idx.iloc[-1] if len(user_items) else None
        return recommend_cold_by_category(last_item, K)
    items, _ = als_weighted.recommend(
        user_idx, train_matrix[user_idx],
        N=K, filter_already_liked_items=True
    )
    return list(items)

cold_cat_scores = evaluate_model(recommend_cold_cat, K=10)
cold_cat_scores


(np.float64(0.011661506707946337),
 np.float64(0.11661506707946337),
 np.float64(0.05968053347123987))

# 🟢 AMÉLIORATION 3 — Re-ranking metadata (post-ALS)

In [125]:
def recommend_rerank_meta(user_idx, K=10):
    if user_interactions.get(user_idx, 0) < 2:
        return recent_items[:K]
    items, _ = als_weighted.recommend(
        user_idx, train_matrix[user_idx],
        N=50, filter_already_liked_items=True
    )
    return rerank_metadata(items, K)

rerank_meta_scores = evaluate_model(recommend_rerank_meta, K=10)
rerank_meta_scores


(np.float64(0.005056759545923633),
 np.float64(0.05056759545923633),
 np.float64(0.029626827888409833))

🟢 AMÉLIORATION 4 — Content-based fallback (TF-IDF)

In [129]:
def recommend_content_fallback(user_idx, K=10):
    # Cold user
    if user_interactions.get(user_idx, 0) < 2:
        user_items = train[train["user_idx"] == user_idx]

        if len(user_items) > 0:
            last_item = user_items.sort_values("ts")["item_idx"].iloc[-1]
            return content_similar_items(last_item, K)

        return recent_items[:K]

    # Warm user → ALS
    items, _ = als_weighted.recommend(
        user_idx,
        train_matrix[user_idx],
        N=K,
        filter_already_liked_items=True
    )
    return list(items)


AMÉLIORATION 5 — Cold + catégorie + reranking (MAXIMAL)

In [127]:
def recommend_maximal(user_idx, K=10):
    if user_interactions.get(user_idx, 0) < 2:
        user_items = train[train.user_idx == user_idx]
        last_item = user_items.item_idx.iloc[-1] if len(user_items) else None
        items = recommend_cold_by_category(last_item, K*2)
        return rerank_metadata(items, K)
    items, _ = als_weighted.recommend(
        user_idx, train_matrix[user_idx],
        N=50, filter_already_liked_items=True
    )
    return rerank_metadata(items, K)

maximal_scores = evaluate_model(recommend_maximal, K=10)
maximal_scores


(np.float64(0.01238390092879257),
 np.float64(0.1238390092879257),
 np.float64(0.07255712923698986))

In [132]:
def content_similar_items(item_idx, K=10):
    # retrouver la ligne correspondant à item_idx
    rows = meta_known.index[meta_known["item_idx"] == item_idx]
    if len(rows) == 0:
        return recent_items[:K]

    idx = rows[0]

    sims = cosine_similarity(X[idx], X)[0]
    top = sims.argsort()[::-1][1:K+1]

    return meta_known.iloc[top]["item_idx"].tolist()


In [133]:
def recommend_content_fallback(user_idx, K=10):
    # Cold user
    if user_interactions.get(user_idx, 0) < 2:
        user_items = train[train["user_idx"] == user_idx]

        if len(user_items) > 0:
            last_item = user_items.sort_values("ts")["item_idx"].iloc[-1]
            return content_similar_items(last_item, K)

        return recent_items[:K]

    # Warm user → ALS
    items, _ = als_weighted.recommend(
        user_idx,
        train_matrix[user_idx],
        N=K,
        filter_already_liked_items=True
    )
    return list(items)


In [135]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1️⃣ Construire le dataframe contenu ALIGNÉ
content_df = meta_known[["item_idx", "title", "brand"]].copy()
content_df["text"] = (
    content_df["title"].fillna("") + " " +
    content_df["brand"].fillna("")
)

# 2️⃣ Reset index (TRÈS IMPORTANT)
content_df = content_df.reset_index(drop=True)

# 3️⃣ TF-IDF
tfidf = TfidfVectorizer(max_features=5000, stop_words="english")
X = tfidf.fit_transform(content_df["text"])

# 4️⃣ Mapping item_idx → row_id
item_to_row = dict(zip(content_df["item_idx"], content_df.index))

# 5️⃣ Fonction content-based SAFE
def content_similar_items(item_idx, K=10):
    if item_idx not in item_to_row:
        return recent_items[:K]

    row = item_to_row[item_idx]
    sims = cosine_similarity(X[row], X)[0]

    # top K similaires (hors lui-même)
    top_rows = sims.argsort()[::-1][1:K+1]

    return content_df.iloc[top_rows]["item_idx"].tolist()


In [136]:
content_scores = evaluate_model(recommend_content_fallback, K=10)
content_scores


(np.float64(0.0086687306501548),
 np.float64(0.08668730650154799),
 np.float64(0.06019833826520749))